In [17]:
import numpy as np
import os
import urllib.request
import zipfile
import imageio

In [18]:
def load_tiny_imagenet_data(data_dir="tiny-imagenet-200"):
    # Download and extract Tiny ImageNet dataset if not already downloaded
    if not os.path.exists(data_dir):
        print("Downloading Tiny ImageNet dataset...")
        url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
        urllib.request.urlretrieve(url, "tiny-imagenet-200.zip")
        with zipfile.ZipFile("tiny-imagenet-200.zip", "r") as zip_ref:
            zip_ref.extractall()

    # Load training data
    train_images = []
    train_labels = []
    train_dir = os.path.join(data_dir, "train")
    for subdir in os.listdir(train_dir):
        subdir_path = os.path.join(train_dir, subdir)
        if os.path.isdir(subdir_path):
            for img_file in os.listdir(subdir_path + "/images"):
                img_path = os.path.join(subdir_path + "/images", img_file)
                img = imageio.imread(img_path)
                train_images.append(img)
                train_labels.append(subdir)

    # Load validation data
    test_images = []
    test_labels = []
    with open(os.path.join(data_dir, "val", "val_annotations.txt"), "r") as file:
        for line in file:
            img_file, label, _, _, _, _ = line.split("\t")
            img_path = os.path.join(data_dir, "val", "images", img_file)
            img = imageio.imread(img_path)
            test_images.append(img)
            test_labels.append(label)

    # Convert lists to numpy arrays
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    return train_images, train_labels, test_images, test_labels

In [32]:
import numpy as np
import os
import urllib.request
import zipfile
from PIL import Image

def load_tiny_imagenet_data(data_dir="tiny-imagenet-200", target_size=(64, 64)):
    # Download and extract Tiny ImageNet dataset if not already downloaded
    if not os.path.exists(data_dir):
        print("Downloading Tiny ImageNet dataset...")
        url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
        urllib.request.urlretrieve(url, "tiny-imagenet-200.zip")
        with zipfile.ZipFile("tiny-imagenet-200.zip", "r") as zip_ref:
            zip_ref.extractall()

    # Load training data
    train_images = []
    train_labels = []
    train_dir = os.path.join(data_dir, "train")
    for subdir in os.listdir(train_dir):
        subdir_path = os.path.join(train_dir, subdir)
        if os.path.isdir(subdir_path):
            for img_file in os.listdir(subdir_path + "/images"):
                img_path = os.path.join(subdir_path + "/images", img_file)
                #img = Image.open(img_path)
                img = img.resize(target_size)  # Resize the image to the target size
                img = np.array(img)
                train_images.append(img)
                train_labels.append(subdir)

    # Load validation data
    test_images = []
    test_labels = []
    with open(os.path.join(data_dir, "val", "val_annotations.txt"), "r") as file:
        for line in file:
            img_file, label, _, _, _, _ = line.split("\t")
            img_path = os.path.join(data_dir, "val", "images", img_file)
            img = Image.open(img_path)
            #img = img.resize(target_size)  # Resize the image to the target size
            img = np.array(img)
            test_images.append(img)
            test_labels.append(label)

    # Convert lists to numpy arrays
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    return train_images, train_labels, test_images, test_labels

In [33]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Resize the images to match the input shape of the model
def resize_images(images):
    resized_images = tf.image.resize(images, (32, 32))
    return resized_images

# Load Tiny ImageNet data
# Assuming you have a function load_tiny_imagenet_data() to load the data
train_images, train_labels, test_images, test_labels = load_tiny_imagenet_data()

# Preprocess the data and resize the images
train_images = resize_images(train_images)
test_images = resize_images(test_images)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(train_images)

# Load the custom model from Experiment 1
# Assuming you have already defined and trained the model
custom_model = model.load(r'/kaggle/input/mycustom/keras/myvalslug/1/cifar.keras')

# Adjust the output layer for 200 classes
custom_model.layers[-1].units = 200
custom_model.layers[-1].activation = 'softmax'

# Unfreeze the base model (VGG19)
custom_model.trainable = True
custom_model.layers[0].trainable = True  # Assuming the base model is the first layer

# Rebuild the model after changing trainable status
custom_model.build((None, 32, 32, 3))

# Define the optimizer with learning rate scheduler
initial_learning_rate = 0.001  # Adjust as needed
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Compile the model with the defined optimizer
custom_model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = custom_model.fit(
    datagen.flow(train_images, train_labels, batch_size=64),
    steps_per_epoch=len(train_images) // 64,
    epochs=10,  # Adjust as needed
    validation_data=(test_images, test_labels)
)

# Report the final training and testing accuracy
final_train_acc = history.history['accuracy'][-1]
final_test_acc = custom_model.evaluate(test_images, test_labels, verbose=0)[1]

print("Final Training Accuracy:", final_train_acc)
print("Final Testing Accuracy:", final_test_acc)

#Optionally, you can save the model and history for further analysis
custom_model.save("custom_model_tiny_imagenet.h5")
np.save("history.npy", history.history)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (100000, 64, 64) + inhomogeneous part.